# RandomForestRegression Year column embedding
    1. Year column embedding code to preserve time series properties.
    2. stock별 시계열패턴을 모델이 학습할수 있도록 작성된 코드.

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# RandomForestRegressor (multi-output 지원)
model = RandomForestRegressor(
    n_estimators=800,
    max_depth=None,           
    min_samples_split=2,
    min_samples_leaf=1,
    n_jobs=-1,                
    random_state=42,
    oob_score=False,
    verbose=1
)
model

,n_estimators,800
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [2]:
# Set target_columns
target_columns = ['CETR', 'GETR', 'TSTA', 'TSDA']

In [3]:
# Load CSV file
df = pd.read_csv('../data/KoTaP_Dataset.csv')
df.shape

(12653, 65)

In [4]:
# Set categorical_cols
categorical_cols = ['name', 'stock', 'KOSPI', 'big4', 'LOSS', 'ind']
for col in categorical_cols:
    df[col] = df[col].astype('category')

print(df.dtypes)

name        category
stock       category
year           int64
KOSPI       category
fnd_year       int64
              ...   
A_CETR       float64
A_GETR3      float64
A_CETR3      float64
A_GETR5      float64
A_CETR5      float64
Length: 65, dtype: object


In [5]:
# Time-series aware encoding & global cutoff split (2011~2023 train, 2024 test)
import numpy as np

# ---- CONFIG ----
STRICT_LAG = True  # True: 이전 해(year-1)가 있는 행만 학습/평가에 사용

# 2. Lag 특징 생성 (1-step) - 이미 존재하는 lag 컬럼은 재생성하지 않음
# 제외 컬럼 정의
exclude_for_lag = set(['name', 'stock', 'year'] + target_columns)

# 이미 존재하는 lag 소스 컬럼 탐지

for c in df.columns:
    if c.startswith('lag1_'):  # 새 패턴: lag1_<col>
        exclude_for_lag.add(c[len('lag1_'):])
        exclude_for_lag.add(c)  # 원본 lag 컬럼명도 추가
    elif c.startswith('lag_'):  # 기존 패턴: lag_<col> (lag__<col> 포함)
        exclude_for_lag.add(c[len('lag_'):])
        exclude_for_lag.add(c)  # 원본 lag 컬럼명도 추가
    else:
        if c.endswith('_lag1') or ('_lag' in c and c.rsplit('_lag', 1)[-1].isdigit()):
            base = c.rsplit('_lag', 1)[0]
            exclude_for_lag.add(base)

# 수치형 후보
num_cols_for_lag = [c for c in df.columns if c not in exclude_for_lag and df[c].dtype.kind in ['i','u','f']]
# 범주형 컬럼도 lag 제외 대상에 추가
exclude_for_lag.update(categorical_cols)

# dtype이 category 또는 object인 컬럼도 추가로 제외
cat_cols_detected = df.select_dtypes(include=['category', 'object']).columns.tolist()
exclude_for_lag.update(cat_cols_detected)

exclude_for_lag.update(target_columns)  # 타겟 컬럼도 제외
print("exclude_for_lag 요소 수:", len(exclude_for_lag))
exclude_for_lag

exclude_for_lag 요소 수: 27


{'CETR',
 'GETR',
 'KOSPI',
 'LOSS',
 'TSDA',
 'TSTA',
 'asset',
 'big4',
 'c_asset',
 'c_liab',
 'equit',
 'ind',
 'lag1_ni',
 'lag_asset',
 'lag_c_asset',
 'lag_c_liab',
 'lag_equit',
 'lag_liab',
 'lag_sales',
 'lag_total',
 'liab',
 'name',
 'ni',
 'sales',
 'stock',
 'total',
 'year'}

In [6]:
# 생성 대상 선정
lag_create_targets = [col for col in num_cols_for_lag if col not in exclude_for_lag]
print("lag생성 column 대상 수:", len(lag_create_targets))
lag_create_targets

lag생성 column 대상 수: 38


['fnd_year',
 'fiscal',
 'forn',
 'own',
 'inv',
 'cogs',
 'dep',
 'tax',
 'rec',
 'ocf',
 'cash',
 'tan',
 'land',
 'cip',
 'intan',
 'pti',
 'SIZE',
 'LEV',
 'CUR',
 'GRW',
 'ROA',
 'ROE',
 'CFO',
 'PPE',
 'AGE',
 'INVREC',
 'MB',
 'TQ',
 'GETR3',
 'CETR3',
 'GETR5',
 'CETR5',
 'A_GETR',
 'A_CETR',
 'A_GETR3',
 'A_CETR3',
 'A_GETR5',
 'A_CETR5']

In [7]:
df.shape
df

,name,stock,year,KOSPI,fnd_year,fiscal,ind,big4,forn,own,...,GETR5,CETR5,TSTA,TSDA,A_GETR,A_CETR,A_GETR3,A_CETR3,A_GETR5,A_CETR5
0,동화약품,20,2013,1,1897,12,21,1,0.0613,0.3114,...,0.294414,0.266121,-0.071863,-0.088363,-0.236479,0.748503,0.058765,-0.096756,0.034497,-0.019779
1,동화약품,20,2014,1,1897,12,21,1,0.0502,0.3151,...,0.233461,0.418783,-0.001226,-0.014614,-0.150710,0.748503,0.500993,0.733432,-0.026455,0.132883
2,동화약품,20,2015,1,1897,12,21,1,0.0749,0.3235,...,0.231593,0.528209,0.072994,0.056283,-0.171420,-0.024525,-0.195255,0.733432,-0.028324,0.242310
3,동화약품,20,2016,1,1897,12,21,1,0.0746,0.3229,...,0.306044,0.325968,0.106756,0.107570,0.026917,-0.194237,-0.174602,0.327249,0.046128,0.040069
4,동화약품,20,2017,1,1897,12,21,1,0.1186,0.3240,...,0.250474,0.216419,-0.061506,0.008744,0.042469,-0.157749,-0.014648,-0.184860,-0.009442,-0.069481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12648,잉글우드랩,950140,2023,0,2015,12,20,0,0.1248,0.4408,...,0.174191,0.211668,-0.317902,-0.362904,-0.122071,0.080095,-0.064306,0.002523,0.001916,0.012473
12649,잉글우드랩,950140,2024,0,2015,12,20,0,0.1358,0.4408,...,0.114758,0.256242,-0.358287,-0.384294,-0.123269,0.292943,-0.129741,0.004858,-0.091928,0.027420
12650,고스트스튜디오,950190,2022,0,2016,12,58,1,0.1374,0.4427,...,0.176417,0.172354,-0.109588,-0.137899,-0.055951,-0.122758,-0.039628,0.121765,-0.064150,-0.092251
12651,고스트스튜디오,950190,2023,0,2016,12,58,1,0.1445,0.5158,...,0.180101,0.204584,-0.225762,-0.237668,-0.034687,-0.052817,-0.041439,-0.021230,-0.060466,-0.060022


In [8]:
# 생성
for col in lag_create_targets:
    df[f'lag__{col}'] = df.groupby('stock', observed=True)[col].shift(1)

print(f"[Lag] lag생성 제외(활용된 col, lag col 그자체 등) 컬럼 수: {len(exclude_for_lag)} | 새로 생성: {len(lag_create_targets)}")
if lag_create_targets:
    print("[Lag] 새로 생성된 예시:", lag_create_targets[:10])

[Lag] lag생성 제외(활용된 col, lag col 그자체 등) 컬럼 수: 27 | 새로 생성: 38
[Lag] 새로 생성된 예시: ['fnd_year', 'fiscal', 'forn', 'own', 'inv', 'cogs', 'dep', 'tax', 'rec', 'ocf']


In [9]:
df.shape
df

,name,stock,year,KOSPI,fnd_year,fiscal,ind,big4,forn,own,...,lag__GETR3,lag__CETR3,lag__GETR5,lag__CETR5,lag__A_GETR,lag__A_CETR,lag__A_GETR3,lag__A_CETR3,lag__A_GETR5,lag__A_CETR5
0,동화약품,20,2013,1,1897,12,21,1,0.0613,0.3114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,동화약품,20,2014,1,1897,12,21,1,0.0502,0.3151,...,0.308501,0.169812,0.294414,0.266121,-0.236479,0.748503,0.058765,-0.096756,0.034497,-0.019779
2,동화약품,20,2015,1,1897,12,21,1,0.0749,0.3235,...,0.750728,1.000000,0.233461,0.418783,-0.150710,0.748503,0.500993,0.733432,-0.026455,0.132883
3,동화약품,20,2016,1,1897,12,21,1,0.0746,0.3229,...,0.054480,1.000000,0.231593,0.528209,-0.171420,-0.024525,-0.195255,0.733432,-0.028324,0.242310
4,동화약품,20,2017,1,1897,12,21,1,0.1186,0.3240,...,0.075133,0.593817,0.306044,0.325968,0.026917,-0.194237,-0.174602,0.327249,0.046128,0.040069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12648,잉글우드랩,950140,2023,0,2015,12,20,0,0.1248,0.4408,...,0.177105,0.251967,0.266716,0.239756,-0.104659,-0.049056,0.001014,0.081624,0.094442,0.040561
12649,잉글우드랩,950140,2024,0,2015,12,20,0,0.1358,0.4408,...,0.111786,0.172867,0.174191,0.211668,-0.122071,0.080095,-0.064306,0.002523,0.001916,0.012473
12650,고스트스튜디오,950190,2022,0,2016,12,58,1,0.1374,0.4427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12651,고스트스튜디오,950190,2023,0,2016,12,58,1,0.1445,0.5158,...,0.178244,0.350704,0.176417,0.172354,-0.055951,-0.122758,-0.039628,0.121765,-0.064150,-0.092251


In [10]:
# 2. Year 기반 전역/종목별 트렌드 & 주기 인코딩
year_min = df['year'].min()
year_max = df['year'].max()

# 전역 정규화 (0~1)
df['year_norm_global'] = (df['year'] - year_min) / (year_max - year_min)

# 종목별 시작 기준 index (0부터 증가)
df['year_index_stock'] = df.groupby('stock', observed=True)['year'].rank(method='dense').astype(int) - 1
# 종목별 상대적 진행도 (0~1)
df['year_norm_stock'] = df['year_index_stock'] / df.groupby('stock', observed=True)['year_index_stock'].transform('max')

# 주기성 (사이클) 표현: 한 전체 구간(2011~2024)을 하나의 2π 주기로 가정
angle = 2 * np.pi * df['year_norm_global']
df['year_sin'] = np.sin(angle)
df['year_cos'] = np.cos(angle)

# 2-1. 실제 연속 연도 여부 계산 (gap 정보)
df['year_gap'] = df.groupby('stock', observed=True)['year'].diff()
df['has_prev_year'] = df['year_gap'].eq(1)  # True/False

df.shape
df

,name,stock,year,KOSPI,fnd_year,fiscal,ind,big4,forn,own,...,lag__A_CETR3,lag__A_GETR5,lag__A_CETR5,year_norm_global,year_index_stock,year_norm_stock,year_sin,year_cos,year_gap,has_prev_year
0,동화약품,20,2013,1,1897,12,21,1,0.0613,0.3114,...,NaN,NaN,NaN,0.153846,0,0.000000,8.229839e-01,0.568065,NaN,False
1,동화약품,20,2014,1,1897,12,21,1,0.0502,0.3151,...,-0.096756,0.034497,-0.019779,0.230769,1,0.090909,9.927089e-01,0.120537,1.0,True
2,동화약품,20,2015,1,1897,12,21,1,0.0749,0.3235,...,0.733432,-0.026455,0.132883,0.307692,2,0.181818,9.350162e-01,-0.354605,1.0,True
3,동화약품,20,2016,1,1897,12,21,1,0.0746,0.3229,...,0.733432,-0.028324,0.242310,0.384615,3,0.272727,6.631227e-01,-0.748511,1.0,True
4,동화약품,20,2017,1,1897,12,21,1,0.1186,0.3240,...,0.327249,0.046128,0.040069,0.461538,4,0.363636,2.393157e-01,-0.970942,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12648,잉글우드랩,950140,2023,0,2015,12,20,0,0.1248,0.4408,...,0.081624,0.094442,0.040561,0.923077,2,0.666667,-4.647232e-01,0.885456,1.0,True
12649,잉글우드랩,950140,2024,0,2015,12,20,0,0.1358,0.4408,...,0.002523,0.001916,0.012473,1.000000,3,1.000000,-2.449294e-16,1.000000,1.0,True
12650,고스트스튜디오,950190,2022,0,2016,12,58,1,0.1374,0.4427,...,NaN,NaN,NaN,0.846154,0,0.000000,-8.229839e-01,0.568065,NaN,False
12651,고스트스튜디오,950190,2023,0,2016,12,58,1,0.1445,0.5158,...,0.121765,-0.064150,-0.092251,0.923077,1,0.500000,-4.647232e-01,0.885456,1.0,True


In [11]:
# Train/test 데이터셋 분리 (validation은 이후 train dataset에서 생성)
# 3. 전역 컷오프 기반 연도 분리 (Train: <=2023, Test: 2024)
CUTOFF_TEST_YEAR = 2024
train_mask_full = df['year'] < CUTOFF_TEST_YEAR
full_test_mask = df['year'] == CUTOFF_TEST_YEAR

train_df = df[train_mask_full].copy()
test_df = df[full_test_mask].copy()


In [12]:
train_df.shape, test_df.shape

((11567, 110), (1086, 110))

In [13]:
# include unseen stocks too by using the union of train and test stocks
seen_stocks = set(train_df['stock'].unique()) | set(test_df['stock'].unique())

# 2024 테스트셋 STRICT_LAG 적용: 직전 연도가 실제로 존재(연속)하는 행만 유지
# removed_train 과 removed_test는 '얼마나 줄었는지' 파악하는 로그용 변수.
if STRICT_LAG:
    before_train_rows = len(train_df)
    before_test_rows = len(test_df)
    train_df = train_df[train_df['has_prev_year']].copy()
    test_df = test_df[test_df['has_prev_year']].copy()
    removed_train = before_train_rows - len(train_df)
    removed_test = before_test_rows - len(test_df)
else:
    removed_train = 0
    removed_test = 0

# untill include target col.
train_df.shape, test_df.shape

((9095, 110), (929, 110))

In [14]:
# 5. Drop target column. X == input
feature_drop = ['name'] + target_columns
X_train = train_df.drop(columns=feature_drop)
X_test_2024 = test_df.drop(columns=feature_drop)
X_train.shape, X_test_2024.shape

((9095, 105), (929, 105))

In [15]:
# y == label.
y_train = train_df[target_columns]
y_test_2024 = test_df[target_columns]
y_train.shape, y_test_2024.shape

((9095, 4), (929, 4))

In [16]:
# 6. Lag 결측 처리: 새 네이밍(lag__*) 포함하여 처리. 기존 'lag_' col은 결측 없음.
lag_cols = [c for c in X_train.columns if c.startswith('lag__') or c.endswith('_lag1')]
if lag_cols:
    lag_medians = X_train[lag_cols].median()
    X_train[lag_cols] = X_train[lag_cols].fillna(lag_medians)
    X_test_2024[lag_cols] = X_test_2024[lag_cols].fillna(lag_medians)

X_train.shape, X_test_2024.shape

((9095, 105), (929, 105))

In [17]:
# 7. Summary
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
print(f"Train years: {int(train_df['year'].min())}-{int(train_df['year'].max())} | Test year: {CUTOFF_TEST_YEAR}")
print(f"Cutoff and Seen stocks count in train_df: {len(seen_stocks)}")
print("STRICT_LAG:", STRICT_LAG, f"| Removed train rows: {removed_train} | Removed test rows: {removed_test}")
print("Shapes -> Train:", X_train.shape, "2024 Test:", X_test_2024.shape)
print("Lag feature count:", len(lag_cols))

Train years: 2012-2023 | Test year: 2024
Cutoff and Seen stocks count in train_df: 1754
STRICT_LAG: True | Removed train rows: 2472 | Removed test rows: 157
Shapes -> Train: (9095, 105) 2024 Test: (929, 105)
Lag feature count: 38


In [18]:
# debug
X_train.to_csv('X_train.csv', index=True)
print('Saved: X_train.csv', X_train.shape)

Saved: X_train.csv (9095, 105)


In [19]:
# 10. 모델 학습 & 평가 (RandomForest)
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

model.fit(X_train, y_train)

pred_seen = model.predict(X_test_2024)
print('[Seen Stocks 2024] R2:', r2_score(y_test_2024, pred_seen),
      'MAE:', mean_absolute_error(y_test_2024, pred_seen),
      'RMSE:', root_mean_squared_error(y_test_2024, pred_seen))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed:   16.5s


[Seen Stocks 2024] R2: 0.9171574239973326 MAE: 0.03336892890352123 RMSE: 0.060885189616991416


[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   17.8s finished
[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=32)]: Done 386 tasks      | elapsed:    0.1s
[Parallel(n_jobs=32)]: Done 736 tasks      | elapsed:    0.1s
[Parallel(n_jobs=32)]: Done 800 out of 800 | elapsed:    0.2s finished


In [20]:
# 타겟별 상세
print('\nPer-target metrics:')
for i, col in enumerate(target_columns):
    r2_i = r2_score(y_test_2024[col], pred_seen[:, i])
    mae_i = mean_absolute_error(y_test_2024[col], pred_seen[:, i])
    rmse_i = root_mean_squared_error(y_test_2024[col], pred_seen[:, i])
    print(f"  {col}: R2={r2_i:.4f} MAE={mae_i:.4f} RMSE={rmse_i:.4f}")

# 중요도
import pandas as pd
fi = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print('\nTop 25 Feature Importances:')
print(fi.head(25))


Per-target metrics:
  CETR: R2=0.9735 MAE=0.0276 RMSE=0.0360
  GETR: R2=0.8587 MAE=0.0309 RMSE=0.0501
  TSTA: R2=0.9184 MAE=0.0368 RMSE=0.0780
  TSDA: R2=0.9181 MAE=0.0382 RMSE=0.0795

Top 25 Feature Importances:
A_CETR              0.281033
SIZE                0.141627
lag_asset           0.124123
year_sin            0.085032
A_GETR              0.080623
asset               0.048914
stock               0.048046
year                0.029271
year_norm_global    0.029031
ocf                 0.021478
lag__SIZE           0.013741
CFO                 0.012504
ROE                 0.010207
ROA                 0.003135
equit               0.003080
ni                  0.002548
lag_equit           0.002166
pti                 0.001969
GETR5               0.001816
lag_liab            0.001809
sales               0.001561
lag__pti            0.001456
GRW                 0.001414
liab                0.001343
lag__CFO            0.001329
dtype: float64


In [ ]:
# 11. 시각화 (Per-target 성능 막대 그래프)
import matplotlib.pyplot as plt

# 타겟별 성능지표 계산
metrics = {'R2': [], 'MAE': [], 'RMSE': []}
for i, col in enumerate(target_columns):
    y_true = y_test_2024[col]
    y_pred = pred_seen[:, i]
    metrics['R2'].append(r2_score(y_true, y_pred))
    metrics['MAE'].append(mean_absolute_error(y_true, y_pred))
    metrics['RMSE'].append(root_mean_squared_error(y_true, y_pred))

metrics_df = pd.DataFrame(metrics, index=target_columns)
print(metrics_df)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for ax, metric in zip(axes, metrics_df.columns):
    metrics_df[metric].plot(kind='bar', ax=ax, color='#4C72B0')
    ax.set_title(metric)
    ax.set_xticklabels(metrics_df.index, rotation=45, ha='right')
    ax.grid(axis='y', linestyle='--', alpha=0.4)
    for p in ax.patches:
        val = p.get_height()
        ax.annotate(f'{val:.3f}', (p.get_x() + p.get_width()/2, val),
                    ha='center', va='bottom', fontsize=8, rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 12. 결과 저장 (Seen test)
import numpy as np, pandas as pd
pred_df = pd.DataFrame(
    data = pred_seen,
    columns = [f'pred_{col}' for col in y_test_2024.columns],
    index = y_test_2024.index
)
actual_df = y_test_2024.add_prefix('actual_')
results_df = pd.concat([actual_df, pred_df], axis=1)
results_df.to_csv('rf_actual_vs_predicted_seen_2024.csv', index=True)
print('Saved: rf_actual_vs_predicted_seen_2024.csv')

# Per-target metrics 저장
metric_rows = []
for i, col in enumerate(target_columns):
    metric_rows.append({
        'target': col,
        'R2': r2_score(y_test_2024[col], pred_seen[:, i]),
        'MAE': mean_absolute_error(y_test_2024[col], pred_seen[:, i]),
        'RMSE': root_mean_squared_error(y_test_2024[col], pred_seen[:, i])
    })
metrics_df = pd.DataFrame(metric_rows)
metrics_df.to_csv('rf_metrics_seen_2024.csv', index=False)
print('Saved: rf_metrics_seen_2024.csv')

## Reference site
    지표 및 점수
    https://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation
    https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
